# 1. 학습 알고리즘 구현하기 (2층 신경망)
# 신경망 학습 절차 
전체 
  신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습' 이라한다. 신경망 학습은 다음과 같은 4단게
  
1단계 - 미니배치 (batch_size, length, |V})

  훈련 데이터 중 일부를 무작위로 가져옴. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실함수 값을 줄이는 것이 목표
  
2단계 - 기울기 산출 

  미니배치의 손실함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수 값을 가장 작게 하는 방향을 제시
  
3단계 - 매개변수 갱신 

  가중치 매개변수를 기울기 방향으로 아주 조금 갱신
  
4단계 - 반복

  1~3 단계 반복

In [3]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from functions import *
from gradient import numerical_gradient


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    
############## 수치미분으로 기울기 구하는 방법

# x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
############## 오차역전법으로 기울기 구하는 방법

#     def gradient(self, x, t):
#         W1, W2 = self.params['W1'], self.params['W2']
#         b1, b2 = self.params['b1'], self.params['b2']
#         grads = {}
        
#         batch_num = x.shape[0]
        
#         # forward
#         a1 = np.dot(x, W1) + b1
#         z1 = sigmoid(a1)
#         a2 = np.dot(z1, W2) + b2
#         y = softmax(a2)
        
#         # backward
#         dy = (y - t) / batch_num
#         grads['W2'] = np.dot(z1.T, dy)
#         grads['b2'] = np.sum(dy, axis=0)
        
#         da1 = np.dot(dy, W2.T)
#         dz1 = sigmoid_grad(a1) * da1
#         grads['W1'] = np.dot(x.T, dz1)
#         grads['b1'] = np.sum(dz1, axis=0)

#         return grads


## np.argmax (x, axis=1) , axis=1 : 열에서 max

# TwoLayerNet 테스트

In [5]:
net = TwoLayerNet(784, 100, 10)

In [7]:
print("net.params['W1'].shape : ", net.params['W1'].shape)
print("net.params['b1'].shape : ", net.params['b1'].shape)
print("net.params['W2'].shape : ", net.params['W2'].shape)
print("net.params['b2'].shape : ", net.params['b2'].shape)

net.params['W1'].shape :  (784, 100)
net.params['b1'].shape :  (100,)
net.params['W2'].shape :  (100, 10)
net.params['b2'].shape :  (10,)


In [8]:
x = np.random.rand(100,784) # 100장 분장의 mini batch (즉 batch_size 가 100)

In [11]:
y = net.predict(x) # 예측값
t = np.random.rand(100,10)

In [12]:
grads = net.numerical_gradient(x,t)

print("grads['W1'].shape : ", grads['W1'].shape)
print("grads['b1'].shape : ", grads['b1'].shape)
print("grads['W2'].shape : ", grads['W2'].shape)
print("grads['b2'].shape : ", grads['b2'].shape)

grads['W1'].shape :  (784, 100)
grads['b1'].shape :  (100,)
grads['W2'].shape :  (100, 10)
grads['b2'].shape :  (10,)


[1] []


# 2. 미니배치 학습 구현하기 

In [ ]:
import numpy as np 
from mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
train_loss_list = []

# 하이퍼파라미터
iters_num = 100 
train_size = x_train.shape[0]
batch_size = 100 # 미니 배치 크기
learning_rate = 0.1

network = TwoLayerNet(784,50,10)



for i in range(iters_num) :
    # 미니배치 획득
    #    print(np.random.choice(500, 3))    ---->   [393 229 471]
    batch_mask = np.random.choice(train_size, batch_size)
    # batch_mask는 train_size 60000까지 숫자 숭에 100개를 랜덤 선택해 list 생성     
    # print(batch_mask) 
    
    # x_batch.shape : (100, 784) 
    # t_batch.shape : (100, 10)
    ########################################################################### 1단계. 미니배치 생성
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    #print(t_batch.shape)
    
    
    ########################################################################### 2단계. 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch)
    
    ########################################################################### 3단계. 매개변수 갱신
    for key in ('W1','b1','W2','b2') :
        network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
        
    

In [34]:
print("x_train.shape   :   ",x_train.shape)
print("x_train[[0]].shape   :   ",x_train[[0]].shape)
print("x_train[[0,1,2,3]]  :  \n", x_train[[0,1,2,3]])

x_train.shape   :    (60000, 784)
x_train[[0]].shape   :    (1, 784)
x_train[[0,1,2,3]]  :  
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
print(train_loss_list)